In [1]:
from collections import deque
from collections import defaultdict
import math
import numpy as np
import os
from operator import attrgetter


In [2]:
filenames = ['data/a.in',
             'data/b.in',
             'data/c.in',
             'data/d.in',
             'data/e.in',
             'data/f.in']
letter_to_int = {'a':0, 'b':1, 'c':2, 'd':3, 'e':4, 'f':5}
int_to_letter = {0 : 'a', 1 : 'b', 2 : 'c', 3 : 'd', 4 : 'f', 5 : 'g'}


letter = 'f'

filename = filenames[letter_to_int[letter]]

In [3]:
class Building:
    
    glob_index = 0
    
    def __init__(self, x: int, y: int, bl: int, bc: int):
        self.x = x
        self.y = y
        self.bl = bl
        self.bc = bc
        self.index = Building.glob_index
        Building.glob_index = Building.glob_index + 1
        self.score = bc - bl
#         self.score = bc
        
class Antenna:
    
    glob_index = 0

    def __init__(self, ar: int, ac: int):
        self.x = -1
        self.y = -1
        self.ar = ar
        self.ac = ac
        self.index = Antenna.glob_index
        Antenna.glob_index = Antenna.glob_index + 1
#         self.score = ac - ar
        self.score = ac

    def assignCoord(self, x, y):
        self.x = x
        self.y = y
        
    def printAsSolution(self):
        return str(self.index) + ' ' + str(self.x) + ' ' + str(self.y)

In [4]:
with open(filename, 'r') as file:
    solution = None
    buildings = []
    antennas = []
    antennas_withRadiusMoreTen = []
    antennas_withoutRadiusMoreTen = []
    for i, line in enumerate(file.readlines()):
        if i == 0:
            [W, H] = line.strip().split(' ')
#                 solution = Solution(int(D),I,S,V,F)
        elif i == 1:
            [N, M, R] = line.strip().split(' ')
            input_coords = np.zeros((int(N), 2))
        elif i <= int(N) + 1:
            # processing buildings
            [bx, by, bl, bc] = line.strip().split(' ')
            build_temp = Building(int(bx), int(by), int(bl), int(bc))

            buildings.append(build_temp)
#             input_coords[build_temp.index] = [int(bx), int(by)]
        else:
            # processing antennas
            [ar, ac] = line.strip().split(' ')

            antenna_temp = Antenna(int(ar), int(ac))
            if int(ar) >= 30:
                antennas_withRadiusMoreTen.append(antenna_temp)
            else:
                antennas_withoutRadiusMoreTen.append(antenna_temp)
            antennas.append(antenna_temp)

In [5]:
# out_antennas = []

# for i in range(int(M)):
#     choosen_building = buildings[i]
#     choosen_antenna = antennas[i]
#     choosen_antenna.assignCoord(choosen_building.x, choosen_building.y)
    
#     out_antennas.append(choosen_antenna)
    
# Building.glob_index = 0
# Antenna.glob_index = 0

In [6]:
# antennas[1654].index
# antennas[200].ar

In [7]:
buildings.sort(key=lambda x: x.score, reverse=True)
antennas.sort(key=lambda x: x.ar, reverse=True)

antennas_withoutRadiusMoreTen.sort(key=lambda x: x.ac, reverse=True)
antennas_withRadiusMoreTen.sort(key=lambda x: x.ar, reverse=True)


points_filled = {}

In [8]:
out_antennas_withoutRange = []
for i in range(len(antennas_withoutRadiusMoreTen)):
    choosen_building = buildings[i]
    buildings.pop(i)
    choosen_antenna = antennas_withoutRadiusMoreTen[i]
    choosen_antenna.assignCoord(choosen_building.x, choosen_building.y)
    points_filled[(choosen_building.x, choosen_building.y)] = 1
    out_antennas_withoutRange.append(choosen_antenna)

In [9]:
for i, build in enumerate(buildings):
    input_coords[i] = [build.x, build.y]
# input_coords


In [10]:
len(antennas_withRadiusMoreTen)

159

In [11]:
from sklearn.cluster import KMeans
import numpy as np

kmeans = KMeans(n_clusters=len(antennas_withRadiusMoreTen), random_state=0, verbose=1, max_iter=25, n_init=3).fit(input_coords)
# kmeans.labels_
# kmeans.predict([[0, 0], [12, 3]])
# kmeans.cluster_centers_

Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 8978842827.976065
start iteration
done sorting
end inner loop
Iteration 1, inertia 8604738350.732546
start iteration
done sorting
end inner loop
Iteration 2, inertia 8468120198.12899
start iteration
done sorting
end inner loop
Iteration 3, inertia 8394696719.944056
start iteration
done sorting
end inner loop
Iteration 4, inertia 8346454443.459491
start iteration
done sorting
end inner loop
Iteration 5, inertia 8310969448.409944
start iteration
done sorting
end inner loop
Iteration 6, inertia 8283794437.315491
start iteration
done sorting
end inner loop
Iteration 7, inertia 8260501217.234139
start iteration
done sorting
end inner loop
Iteration 8, inertia 8239927655.129063
start iteration
done sorting
end inner loop
Iteration 9, inertia 8221740979.333094
start iteration
done sorting
end inner loop
Iteration 10, inertia 8205423156.124057
start iteration
done sorting
end inner loop
Iteration 11, inert

In [12]:
# kmeans.labels_
points_filled.get((321, 299))
# (321,299) in points_filled

In [13]:
out_antennas_withRange = []

assigned_centroids = {}
for i in range(len(buildings)):
    choosen_building = buildings[i]
    centroid = kmeans.cluster_centers_[kmeans.labels_[i]]
    coord = (int(round(centroid[0], 0)), int(round(centroid[1], 0)))
#     print(coord)
    if kmeans.labels_[i] not in assigned_centroids:
        if coord in points_filled:
            if (coord[0]+1,coord[1]) not in points_filled:
                coord = (coord[0]+1,coord[1])
            elif (coord[0]-1,coord[1]) not in points_filled:
                coord = (coord[0]-1,coord[1])
            elif (coord[0],coord[1]+1) not in points_filled:
                coord = (coord[0],coord[1]+1)
            elif (coord[0],coord[1]-1) not in points_filled:
                coord = (coord[0],coord[1]-1)
            else:
                print('non ho trovato niente')
        
        if coord not in points_filled:
            assigned_centroids[kmeans.labels_[i]] = 0
            points_filled[coord] = 1
            choosen_antenna = antennas_withRadiusMoreTen[len(out_antennas_withRange)]
            choosen_antenna.assignCoord(coord[0], coord[1])
            out_antennas_withRange.append(choosen_antenna)
        else:
            pass
            print('Ancora già preso')
            
    if len(out_antennas_withRange) == len(antennas_withRadiusMoreTen):
        break


In [14]:
out_string = str(len(out_antennas_withRange) + len(out_antennas_withoutRange)) + '\n'
for antenna in out_antennas_withRange:
    out_string += antenna.printAsSolution() + '\n'
for antenna in out_antennas_withoutRange:
    out_string += antenna.printAsSolution() + '\n'
out_string = out_string[:-1]

In [15]:
# print(out_string)

In [16]:
filename = 'out/out_'+letter+'.txt'
with open(filename, 'a') as out:
    print(out_string, file = out)